# Setup

In [1]:
import os
import pandas as pd

In [2]:
# data paths:
ORIG_SPLIT_METADATA_FILES = {
    "unbalanced_train": "/proj/systewar/datasets/audioset_music_mood/metadata_unbalanced_train.csv",
    "eval": "/proj/systewar/datasets/audioset_music_mood/metadata_eval.csv"
}

In [3]:
# script options:
N_class_test = 100     # number of samples per class in (new) test set
new_split_metadata_dir = "/proj/systewar/datasets/audioset_music_mood/new_split_metadata_files"
new_split_metadata_files = {
    "train": "/proj/systewar/datasets/audioset_music_mood/new_split_metadata_files/metadata_train.csv",
    "test": "/proj/systewar/datasets/audioset_music_mood/new_split_metadata_files/metadata_test.csv"
}

# Split Dataset

In [4]:
# load original split metadata files:
orig_split_metadata_dfs = {}
for subset, file_path in ORIG_SPLIT_METADATA_FILES.items():
    print("Loading {} set labels...".format(subset))
    orig_split_metadata_dfs[subset] = pd.read_csv(file_path)

# concatenate original split labels into a single dataframe:
all_metadata = pd.concat(orig_split_metadata_dfs.values(), axis="index")
all_metadata = all_metadata.reset_index(drop=True)
print()
print(all_metadata.info())

Loading unbalanced_train set labels...
Loading eval set labels...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13395 entries, 0 to 13394
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   orig_subset     13395 non-null  object
 1   file_name       13395 non-null  object
 2   length_samples  13395 non-null  int64 
 3   label           13395 non-null  object
dtypes: int64(1), object(3)
memory usage: 418.7+ KB
None


In [5]:
# group by emotion class label:
all_metadata_groups = all_metadata.groupby(by="label", axis="index")

# construct stratified test set by randomly sampling from each class:
metadata_test = all_metadata_groups.sample(n=N_class_test, random_state=42)
print(metadata_test.info())

# sanity check:
for count in metadata_test["label"].value_counts():
    assert count == N_class_test, "Error with creating stratified test set."

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 10980 to 3237
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   orig_subset     700 non-null    object
 1   file_name       700 non-null    object
 2   length_samples  700 non-null    int64 
 3   label           700 non-null    object
dtypes: int64(1), object(3)
memory usage: 27.3+ KB
None


In [6]:
# construct training set:
metadata_train = all_metadata.drop(index=list(metadata_test.index))
print(metadata_train.info())
print()
print(metadata_train["label"].value_counts())

# sanity check:
assert set(metadata_train.index).isdisjoint(set(metadata_test.index)), "Train and test sets are not disjoint."

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12695 entries, 0 to 13394
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   orig_subset     12695 non-null  object
 1   file_name       12695 non-null  object
 2   length_samples  12695 non-null  int64 
 3   label           12695 non-null  object
dtypes: int64(1), object(3)
memory usage: 495.9+ KB
None

Exciting music    4431
Tender music      3214
Scary music       1253
Sad music         1242
Happy music       1006
Angry music        794
Funny music        755
Name: label, dtype: int64


In [7]:
# reset indices:
metadata_train = metadata_train.reset_index(drop=True)
metadata_test = metadata_test.reset_index(drop=True)

# more sanity checks:
assert all_metadata.shape[0] == metadata_train.shape[0] + metadata_test.shape[0], "Train and test set sizes don't add up."
assert set(metadata_train["file_name"].tolist()).isdisjoint(set(metadata_test["file_name"].tolist())), "Train and test sets are not disjoint."
class_counts_all = all_metadata["label"].value_counts()
class_counts_train = metadata_train["label"].value_counts()
class_counts_test = metadata_test["label"].value_counts()
for class_label in all_metadata_groups.groups.keys():
    assert class_counts_all[class_label] == class_counts_train[class_label] + class_counts_test[class_label], "Error with splitting dataset."

# save to files:
os.makedirs(new_split_metadata_dir, exist_ok=True)
metadata_train.to_csv(new_split_metadata_files["train"], index=False)
metadata_test.to_csv(new_split_metadata_files["test"], index=False)